In [ ]:
# written by Keisuke Hinode（hinode@rikkyo.ac.jp）
# See also https://github.com/Keisuke-Hinode/NLP-Sentiment-Analysis 

In [ ]:
# テキストファイルをアップロードする
from google.colab import files
files.upload()

In [ ]:
#都合により、file_data1と2で同じものを指定
file_data1 = open("text.csv","r",encoding="utf-8")
file_data2 = open("text.csv","r",encoding="utf-8")

In [ ]:
# MeCabのインストールする
!apt install mecab libmecab-dev mecab-ipadic-utf8
!pip install mecab-python3

In [ ]:
# mecab-ipadic-NEologdのインストールする
!apt install git make curl xz-utils file
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a
!ln -s /etc/mecabrc /usr/local/etc/mecabrc

In [ ]:
# ML-Askをインストールする
# Python版の実装（pymlask）を使う
!pip install  pymlask

ポジティブ・ネガティブ分析

In [ ]:
# 参考：https://qiita.com/masahiro54/items/e2186ec7ea8d91b84187
# および：https://qiita.com/hnishi/items/0d32a778e375a99aff13
import mlask
import subprocess

cmd='echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                           shell=True).communicate()[0]).decode('utf-8')

In [ ]:
emotion_analyzer = mlask.MLAsk('-d {0}'.format(path))
#初期化
senryu =[]
positive_count = 0
negative_count = 0
neutral_count = 0
exception_count = 0
for line in file_data1:
    #改行コードを削除して、感情分析を行います。
    emotion_senryu = emotion_analyzer.analyze(line.replace("\n",""))
    #orientationが設定されない場合があるので、try exceptで囲っています。
    try:
        if  "POSITIVE" in emotion_senryu['orientation']:
            positive_count += 1
            #print("POSITIVE") ←デバッグ用です。
            #print(line)
        elif "NEGATIVE" in emotion_senryu['orientation']:
            negative_count += 1
            #print("NEGATIVE")
            #print(line)
        else:
            neutral_count += 1
    #判定不能の場合はexception_countに割り振ります。
    except:
        exception_count += 1
#トータルの件数
print("ネガティブ：" + str(negative_count))
print("ポジティブ：" + str(positive_count))
print("ニュートラル：" + str(neutral_count))
print("例外："+str(exception_count))

#10万行で約2分30秒（GPU不使用）
#80万行で約12分(GPU使用)

10分類

入力されたテキストに10種類の感情を割り当てる

喜・怒・昂・哀・好・怖・安・厭・驚・恥

ML-Askによる10分類の感情分析を用いた先行研究は、以下の2つが比較的わかりやすい

[鳥海・榊・吉田（2020）](https://www.jstage.jst.go.jp/article/tjsai/35/4/35_F-K45/_pdf)

[笹原・奥田・五十嵐（2021）](https://www.jstage.jst.go.jp/article/pjsai/JSAI2021/0/JSAI2021_1D3OS3b04/_pdf)

どちらも、分類された感情ごとにTF-IDFで特徴語を算出し、その感情をもたらした要因を考察している



1. まずは10分類の数を出す

In [ ]:
import mlask
import subprocess

cmd='echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                           shell=True).communicate()[0]).decode('utf-8')

In [ ]:
emotion_analyzer = mlask.MLAsk('-d {0}'.format(path))
#初期化
senryu=[]
iya_count = 0
yorokobi_count = 0
yasu_count = 0
suki_count = 0
aware_count = 0
ikari_count = 0
odoroki_count = 0
takaburi_count = 0
haji_count = 0
kowa_count = 0
exception_count = 0
for line in file_data2:
 
    emotion_senryu = emotion_analyzer.analyze(line.replace("\n",""))

    try:
        if  'iya' in emotion_senryu ['representative']:
            iya_count += 1
        
        elif 'yorokobi' in emotion_senryu ['representative']:
            yorokobi_count += 1

        elif 'yasu' in emotion_senryu ['representative']:
            yasu_count += 1
        
        elif 'suki' in emotion_senryu ['representative']:
            suki_count += 1
        
        elif 'aware' in emotion_senryu ['representative']:
            aware_count += 1
        
        elif 'ikari' in emotion_senryu ['representative']:
            ikari_count += 1
        
        elif 'odoroki' in emotion_senryu ['representative']:
            odoroki_count += 1
        
        elif 'takaburi' in emotion_senryu ['representative']:
            takaburi_count += 1
        
        elif 'haji' in emotion_senryu ['representative']:
            haji_count += 1

        elif 'kowa' in emotion_senryu ['representative']:
            kowa_count += 1

    #判定不能の場合はexception_countに割り振ります。
    except:
        exception_count += 1
#トータルの件数
print("好：" + str(suki_count))
print("喜：" + str(yorokobi_count))
print("安：" + str(yasu_count))
print("昂：" + str(takaburi_count))
print("驚：" + str(odoroki_count))
print("怒：" + str(ikari_count))
print("厭：" + str(iya_count))
print("恥：" + str(haji_count))
print("哀：" + str(aware_count))
print("怖：" + str(kowa_count))
print("例外："+str(exception_count))



2. 次に、それぞれのテキストに感情を割り当てていく

In [ ]:
#pandasを使う
#予め"（ダブルクオーテーション）をエディタで削除してからでないと、1行1投稿にならないことがあるので注意

import pandas as pd
text = pd.read_csv("text.csv")

In [ ]:
print(len(text))

In [ ]:
text.head()

In [ ]:
import mlask
import subprocess

cmd='echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                           shell=True).communicate()[0]).decode('utf-8')
                           
emotion_analyzer = mlask.MLAsk('-d {0}'.format(path))

In [ ]:
#動作テスト
emotion_analyzer.analyze("彼のことは嫌いではない")

In [ ]:
#このように、出力される結果が見にくいので、以下のように処理

In [ ]:
#applymapで1行ずつ処理をしていく
#結果を新しいデータフレームに格納
result = text.applymap(emotion_analyzer.analyze)


#処理時間はこれまでと同程度

In [ ]:
print(len(result))

In [ ]:
result.head()

In [ ]:
#以下、半ば強引にデータフレームの中で結果を整形する
result = result.rename(columns={'内容': 'emotion'})

In [ ]:
result['emotion'] = result['emotion'] .astype(str)

In [ ]:
result["emotion2"] = result["emotion"].replace("(.*)(?=emotion)", "", regex=True)

In [ ]:
result.head()

In [ ]:
result["emotion2"] = result["emotion2"].replace("(?<=})(.*)", "", regex=True)
result["emotion2"] = result["emotion2"].str.replace("}$", "", regex=True)
result["emotion2"] = result["emotion2"].str.replace('{', '', regex=True)
result["emotion2"] = result["emotion2"].replace('[!"#$%&\'\\\\()*+,-./:;<=>?@\\^_`{|}~「」〔〕“”〈〉『』【】＆＊・（）＄＃＠。、？！｀＋￥％]', "", regex=True)

In [ ]:
result.head()

In [ ]:
#ここで、もとの投稿が入ったデータフレーム（text）と結果のデータフレーム（result[emotion2]）にそれぞれ行の番号を振って、それをキーに結合する
serial_num = pd.RangeIndex(start=1, stop=len(text.index) + 1, step=1)
text['No'] = serial_num
serial_num = pd.RangeIndex(start=1, stop=len(result.index) + 1, step=1)
result['No'] = serial_num

In [ ]:
analysis = pd.merge(text, result[["emotion2", "No"]], on="No", how="outer")
analysis.head()

In [ ]:
del analysis["No"]
analysis = analysis.rename(columns={'emotion2': 'emotion'})

In [ ]:
analysis.head()

In [ ]:
#感情無しの行は行ごと削除しても良い
#analysis = analysis[~analysis['emotion'].str.contains('感情無し')]

#emotion列の不要な部分を消す
analysis['emotion'] = analysis['emotion'].str.replace('emotion defaultdictclass list', '', regex=True)

In [ ]:
#結果の感情成分を漢字に変換
analysis['emotion'] = analysis['emotion'].str.replace('suki','好', regex=True)
analysis['emotion'] = analysis['emotion'].str.replace('yorokobi','喜', regex=True)
analysis['emotion'] = analysis['emotion'].str.replace('yasu','安', regex=True)
analysis['emotion'] = analysis['emotion'].str.replace('takaburi','昂', regex=True)
analysis['emotion'] = analysis['emotion'].str.replace('odoroki','驚', regex=True)
analysis['emotion'] = analysis['emotion'].str.replace('ikari','怒', regex=True)
analysis['emotion'] = analysis['emotion'].str.replace('iya','厭', regex=True)
analysis['emotion'] = analysis['emotion'].str.replace('haji','恥', regex=True)
analysis['emotion'] = analysis['emotion'].str.replace('aware','哀', regex=True)
analysis['emotion'] = analysis['emotion'].str.replace('kowa','怖', regex=True)

In [ ]:
#最終結果
analysis.head()

In [ ]:
analysis.to_csv("result.csv", index=False)

課題


*   精度があまり高くない（3割くらいしか分類出来ない）
*   ベースの辞書（[『感情表現辞典』](https://www.amazon.co.jp/%E6%84%9F%E6%83%85%E8%A1%A8%E7%8F%BE%E8%BE%9E%E5%85%B8-%E4%B8%AD%E6%9D%91-%E6%98%8E/dp/4490103395)）が古い


※ どの感情表現がどの分類に対応するか、[感情表現の一覧を作者のHPからcsvでダウンロードできる](http://arakilab.media.eng.hokudai.ac.jp/~ptaszynski/repository/mlask.htm)





しかし

*  ポジネガや10分類で結果がわかりやすい
*  感情分析（感情推定）自体、難しい技術であり、現状使いやすいものが少ない（[参考：鳥海さん「感情抽出は技術的に非常に難しい」](https://news.yahoo.co.jp/byline/toriumifujio/20200720-00188646)）
*  単純に10分類するのでは無く、分類した上で時系列での変化や比較をすれば、より訴求力のある結果になる
*  もとの感情表現のcsvを自分で更新すれば、アレンジした分析ができる（Pythonで実装するのが難しくなるが）




